### Часть 1: Определение схемы контекста

In [35]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.messages import HumanMessage
import os

load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")


llm = ChatOpenAI(
    model="deepseek/deepseek-v3.2",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)

- **ColourContext** — это структура данных, описывающая, какую информацию о пользователе мы хотим передать агенту.
- Используем @dataclass — это стандартный способ в Python создавать лёгкие классы для хранения данных.
- Поля имеют значения по умолчанию (blue, yellow), но их можно переопределить.

В реальности это может быть UserContext с user_id, language, subscription_tier, company_name и т.д.

In [36]:
from dataclasses import dataclass

@dataclass
class ColourContext:
    favourite_colour: str = "blue"
    least_favourite_colour: str = "yellow"

### Часть 2: Агент с контекстом (без инструментов)

In [37]:
agent = create_agent(
    model=llm,
    context_schema=ColourContext
)

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite color?")]},
    context=ColourContext()
)

print(response['messages'][-1].content)

I don’t have access to personal information about you unless you share it with me. If you’d like, you can tell me your favorite color, and I can help you with related ideas, facts, or creative suggestions!


In [38]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is my favourite color?', additional_kwargs={}, response_metadata={}, id='9f81e4eb-1baa-4451-a2fc-ab493538f903'),
              AIMessage(content='I don’t have access to personal information about you unless you share it with me. If you’d like, you can tell me your favorite color, and I can help you with related ideas, facts, or creative suggestions!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 16, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 2.24e-05, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 4.16e-06, 'upstream_inference_completions_cost': 1.824e-05}}, 'model_provider': 'openai', 'mode

### Часть 3: Доступ к контексту через инструменты (ToolRuntime)

In [39]:
from langchain.tools import tool, ToolRuntime

@tool
def get_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the favourite colour of the user"""
    return runtime.context.favourite_colour

@tool
def get_least_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the least favourite colour of the user"""
    return runtime.context.least_favourite_colour

In [40]:
agent = create_agent(
    model=llm,
    tools=[get_favourite_colour, get_least_favourite_colour]
)

In [41]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

print(response['messages'][-1].content)

c:\Users\ryash\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [input_value=ColourContext(favourite_c...vourite_colour='yellow'), input_type=ColourContext])
  return self.__pydantic_serializer__.to_python(


Your favourite colour is blue.


In [42]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext(favourite_colour="green")
)

print(response['messages'][-1].content)

Your favourite colour is green.
